# Transfer Learning with RAG Models
----
**Objective**: In this notebook, you will experiment with a QA RAG model on the climate_fever dataset using the Haystack framework. You will get to go through the whole process of fitting the parts of a RAG model together, and learn how to prompt it with queries to get answers from the provided dataset.

NOTE: Make sure to change the runtime from CPU to TPU or GPU for faster training

## Install Libraries
Install the Haystack (for colab) and Datasets libraries

In [ ]:
!pip install farm-haystack[colab]
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.0/856.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 25.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.19.4 which is incompatible.


## Import Dataset
----
In this section, we will use as an example the climate_fever dataset. The dataset consists of 1535 rows of claims about climate change, and they either refute or support climate change, with some claims being neutral. We will build with a specific topic in mind so that we can get more accurate answers, and keep in mind that bigger datasets with open topics can also be used.

Using the "load_dataset" function to load the "climate_fever" dataset with the "test" split.

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('climate_fever', split='test')

## Formatting and Writing Documents
----
First, we need to extract, format and write the documents from our chosen dataset so that we can later build our QA Pipeline. This Pipeline will facilitate the process of building our RAG model and getting answers from it.

Keep in mind that for this notebook we will focus on how to build the pipeline with the simplest configurations. Feel free to experiment with different parameters.



Using the write_documents method to save the formatted documents into document_storage

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

# Extract and format the documents from the dataset
documents = [{"content": str(text)} for text in dataset['claim']]


document_storage = InMemoryDocumentStore(use_bm25=True)

# Write the documents to the document store
document_storage.write_documents(documents)


Updating BM25 representation...: 100%|██████████| 1535/1535 [00:00<00:00, 101806.71 docs/s]


## Preparing the Retriever
----
We need to prepare our Retriever node of our pipeline. It will be responsible to get the documents from our document storage, so that they can be used by the Language Model later. We will the BM25Retriever provided by haystack, as it is the recommended Retriever for begginners.

Create the BM25Retriever using the document_storage created earlier, with a top_k of value 2

In [ ]:
from haystack.nodes import BM25Retriever

# Note: The higher the top_k is, the better the answer will be. However, speed will be affected
retriever = BM25Retriever(document_store=document_storage, top_k=2)


## Preparing the Language Model
----
Now, we will prepare our Language Model using the prompt node. We need to first create our prompt, and for that, Haystack requires a specific structure. We will then define our desired language model alongside the prompt template we created. When creating this template, we need to Parse the output to a format that Haystack can use.

Define the prompt node using PromptNode with the model name as "google/flan-t5-large" and the default prompt template as the created "rag_prompt"

In [ ]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

rag_prompt = PromptTemplate(
    prompt="""Create comprehensive answers from the related text given the questions.
                             Provide a clear and concise response that displays the key points and information presented in the related text.
                             Your answer should be in your own words and be no longer than 50 words.
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser(),
)

prompt_node = PromptNode(model_name_or_path = "google/flan-t5-large", default_prompt_template = rag_prompt)

## Fitting our Pipeline Together


Add the retriever node and prompt_node created in the previous steps to the Pipeline using the add_node function. Hint: you need to provide the inputs to each of these nodes.

In [ ]:
from haystack.pipelines import Pipeline

pipe = Pipeline()
pipe.add_node(retriever, name="retriever", inputs=["Query"])
pipe.add_node(prompt_node,  name="prompt_node", inputs=["retriever"])

## Asking the RAG Model Questions
----
We use the pipeline .run() method to ask a question. Since the output provided by our Prompt Node is a Haystack object, we retrieve in the way provided inside the print() function.

In [ ]:
output = pipe.run(query="When did global warming start")

print(output["answers"][0].answer)

The 20th century global warming did not start until 1910.


In [ ]:
# Here are some other examples you can use
examples = [
    "Who is most responsible for pollution",
    "What is the biggest damaging factor for the climate?",
    "What are some clean energy sources?",
    "How much does the average temperature of our planet rise per decade?"
]